In [6]:
import numpy as np
import hail as hl

from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()

import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain
import statistics as stat
from collections import Counter
    
import bokeh.palettes

from bokeh.plotting import figure, show, output_notebook
output_notebook()

Loading BokehJS ...

Loading BokehJS ...

In [2]:
hl.init(
    tmp_dir='/net/ascratch/people/plggosborcz/gosborcz-hail',
    spark_conf={'spark.driver.memory': '30G', 'spark.executor.memory': '30G'},
    default_reference='GRCh38') 

Loading BokehJS ...

Loading BokehJS ...

2023-01-12 20:28:40.894 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://ac0678:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.105-acd89e80c345
LOGGING: writing to /net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/analysis/oligogenic-model/hail-20230112-2028-0.2.105-acd89e80c345.log


In [ ]:
mt_test = hl.read_matrix_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/GTS-gnomad-sex.mt'
)

pheno = hl.import_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/pheno/GTS-coded-corrected-june-2021.csv',
    impute=True,
    delimiter=',',
    quote="\""
)

pheno = pheno.key_by(pheno.ID)

mt_test = mt_test.annotate_cols(phenotypes = pheno[mt_test.s])
mt_test = mt_test.annotate_cols(category = hl.if_else(mt_test.s.contains('gnomad'), False, (mt_test.phenotypes.disease == 'YES')))
mt_test = mt_test.filter_cols(mt_test.phenotypes.family == '.', keep = False)
mt_test = mt_test.filter_cols((mt_test.phenotypes.disease == 'YES') | (mt_test.phenotypes.disease == 'NO'))
mt_test = mt_test.filter_rows(mt_test.cadd > 0) 
mt_test = mt_test.naive_coalesce(500)
mt_test.write(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt-for-test.mt'
)

mt_discovery = hl.read_matrix_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt-for-model-expanded-may.mt'
) #there is a mistake in phenotype annotations here - it is corrected below

mt_discovery = mt_discovery.annotate_cols(
    category = hl.if_else(
        mt_discovery.s.contains('gnomad'),
        False,
        True
    )
)
mt_discovery = mt_discovery.naive_coalesce(500)
mt_discovery.write(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt-discovery.mt'
)

In [3]:
mt_discovery = hl.read_matrix_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt-discovery.mt'
)
    
mt_test = hl.read_matrix_table(
    '/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/mts/oligogenic-model/mt-for-test.mt'
)

categories = mt_discovery.category.collect()
categories_test = mt_test.category.collect()

In [4]:
top_genes = ['HDC', 'CHADL', 'MAOA', 'NAA11', 'PCDH10']
   
variants_controls = np.zeros((len(top_genes)))
variants_gts = np.zeros((len(top_genes)))
    
variants_controls_test = np.zeros((len(top_genes)))
variants_gts_test = np.zeros((len(top_genes)))

model_asignment = np.zeros((len(top_genes), 80))
test_asignment = np.zeros((len(top_genes), 145)) 
    
for rows, n in enumerate(top_genes):
    mt_discovery_genes = mt_discovery.filter_rows(mt_discovery.nearest_genes_20kb.contains(n)) # to ma gnomadów i heavy tics
    mt_test_genes = mt_test.filter_rows(mt_test.nearest_genes_20kb.contains(n)) # to ma rodziny

    mt_discovery_genes = mt_discovery_genes.filter_rows(mt_discovery_genes.cadd > 10)
    mt_discovery_genes = mt_discovery_genes.annotate_cols(
        non_refs = hl.agg.count_where(mt_discovery_genes.GT.is_non_ref())
    ) #count variants per sample of gnomads and heavy tics  
    non_refs = mt_discovery_genes.non_refs.collect()

    mt_test_genes = mt_test_genes.filter_rows(mt_test_genes.cadd > 10)
    mt_test_genes = mt_test_genes.annotate_cols(
        non_refs = hl.agg.count_where(mt_test_genes.GT.is_non_ref())
    ) #count variants per sample, prepare also test matrix
    non_refs_test = mt_test_genes.non_refs.collect()

    variants_gts[rows] = np.mean(np.array(non_refs)[categories])
    variants_controls[rows] = np.mean(np.array(non_refs)[np.invert(categories)])

    variants_gts_test[rows] = np.mean(np.array(non_refs_test)[categories_test])
    variants_controls_test[rows] = np.mean(np.array(non_refs_test)[np.invert(categories_test)])

    results = (non_refs - variants_controls[rows]) 
    results_test = (non_refs_test - variants_controls[rows]) 

    model_asignment[rows] = results
    test_asignment[rows] = results_test

model_asignment = np.sum(model_asignment, axis = 0)
test_asignment = np.sum(test_asignment, axis = 0)
            
np.save('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/oligogenic-auc/variants_gts.npy', variants_gts)
np.save('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/oligogenic-auc/variants_controls.npy', variants_controls)

np.save('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/oligogenic-auc/variants_gts_test.npy', variants_gts_test)
np.save('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/oligogenic-auc/variants_controls_test.npy', variants_controls_test)

np.save('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/oligogenic-auc/model_asignment.npy', model_asignment)
np.save('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/oligogenic-auc/test_asignment.npy', test_asignment)

false_pos = []
true_pos = []

for x in np.linspace(-25,20,20000):
    false_pos.append(np.sum((test_asignment > x)[np.invert(categories_test)])/56)
    true_pos.append(np.sum((test_asignment > x)[categories_test])/89)
             
np.save('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/oligogenic-auc/false_pos_test.npy', false_pos)
np.save('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/oligogenic-auc/true_pos_test.npy', true_pos)

false_pos = []
true_pos = []

for x in np.linspace(-25,20,20000):
    false_pos.append(np.sum((model_asignment > x)[np.invert(categories)])/40)
    true_pos.append(np.sum((model_asignment > x)[categories])/40)
                
np.save('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/oligogenic-auc/false_pos_discovery.npy', false_pos)
np.save('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/oligogenic-auc/true_pos_discovery.npy', true_pos)
            
print(variants_gts)
print(variants_controls)

[ 1.85   3.875  0.275  5.675 10.875]
[ 2.25   3.325  0.15   6.65  10.25 ]


In [7]:
false_pos = []
true_pos = []
false_pos = np.load('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/oligogenic-auc/false_pos_test.npy')
true_pos = np.load('/net/pr2/projects/plgrid/plggneuromol/imdik-zekanowski-gts/data/oligogenic-auc/true_pos_test.npy')

In [16]:
from bokeh.models import Label, LabelSet

colors = bokeh.palettes.Category20[16]

y = np.linspace(0,1,10)
x = np.linspace(0,1,10)

from bokeh.plotting import figure, output_notebook, show

output_notebook

p = figure(plot_width=800, plot_height=800)
p.line(x, y, line_width=4, line_color='lightgrey')

p.line(false_pos, true_pos, line_width=4, line_color='orange')

#p.circle(false_pos[10740], true_pos[10740], size=12, color="darkred", alpha=1)
p.circle(false_pos[10760], true_pos[10760], size=12, color="red", alpha=1)

#labels = LabelSet(x=[false_pos[10740], false_pos[10760]], y=[false_pos[10740], false_pos[10760]], text=['point A', 'point B'],
#              x_offset=5, y_offset=5, render_mode='canvas')

#p.add_layout(labels)


p.xaxis.axis_label = 'false positives'
p.yaxis.axis_label = 'true positives'

p.xaxis.axis_label_text_font_size = "15pt"
p.xaxis.major_label_text_font_size = "15pt"
p.yaxis.axis_label_text_font_size = "15pt"
p.yaxis.major_label_text_font_size = "15pt"

show(p)

In [20]:
false_pos[10760]

0.44642857142857145

In [18]:
true_pos[10760]

0.651685393258427

In [22]:
-np.trapz(true_pos, false_pos)

0.5968097913322632

In [23]:
np.linspace(-25,20,20000)[10760] #this is the threshold

-0.7887894394719766

In [34]:
np.sum((test_asignment>np.linspace(-25,20,20000)[10760]) == [categories_test][0])

89

In [ ]:
np.where(bank['Gender'] == "Female", 1, 0)

https://datascience.oneoffcoder.com/psuedo-r-squared-logistic-regression.html

In [42]:
import sys
!{sys.executable} -m pip install -U scikit-learn

  Using cached scikit_learn-1.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.5 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)


In [94]:
from sklearn.linear_model import LogisticRegression

y=np.array(categories_test)
X=test_asignment.reshape(-1, 1)
lr = LogisticRegression(fit_intercept=False, solver='lbfgs')
lr.fit(X,y)

w = np.array(lr.coef_).transpose()
y_pred = lr.predict_proba(test_asignment.reshape(-1, 1))[:, 1]

print(lr.coef_)

[[0.02003426]]


In [95]:
from sklearn.metrics import average_precision_score, brier_score_loss, \
    log_loss, roc_auc_score
from sklearn.metrics import explained_variance_score, mean_absolute_error, \
    mean_squared_error, median_absolute_error, r2_score

m_names = [
    'average_precision_score', 'brier_score_loss',
    'log_loss', 'roc_auc_score',
    'explained_variance_score', 'mean_absolute_error',
    'mean_squared_error', 'median_absolute_error', 'r2_score'
]
metrics = [average_precision_score, brier_score_loss, log_loss, roc_auc_score,
           explained_variance_score, mean_absolute_error, mean_squared_error, median_absolute_error, r2_score]
for n, m in zip(m_names, metrics):
    print('{:.5f} : {}'.format(m(y, y_pred), n))

0.71741 : average_precision_score
0.24936 : brier_score_loss
0.69193 : log_loss
0.59681 : roc_auc_score
0.00619 : explained_variance_score
0.49875 : mean_absolute_error
0.24936 : mean_squared_error
0.49812 : median_absolute_error
-0.05193 : r2_score
